# Capstone Project

In [433]:
# Primary Libraries
import urllib, json
import urllib3
import requests
import pandas as pd
from io import StringIO
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# settings to display all columns
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

# Clustering Libraries
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import calinski_harabasz_score, adjusted_rand_score

# Linear Regression stuff
from scipy import stats
from sklearn.model_selection import train_test_split, cross_val_score, KFold
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols

import warnings
warnings.filterwarnings("ignore")

In [484]:
# Function for one-hot encoding
def one_h_e(df, categories, drop_first = True):
    '''Enter datframe and the columns you want to one hot encode'''
    
    df_ohe = df[categories].astype('category')
    
    df_ohe = pd.get_dummies(df_ohe, drop_first = drop_first)
    
    df_done = pd.concat([df, df_ohe], axis = 1)
    
    df_done = df_done.drop(categories, axis = 1)
    
    return df_done

In [490]:
clean_df = pd.read_csv('data/clean_df.csv')

In [491]:
model_df = clean_df.copy()

model_df.dropna(inplace=True)

model_stocks = model_df[['Symbol', 'Name', 'Description']]

model_df.drop(['Symbol', 'Name', 'Description', 'Unnamed: 0'], inplace=True, axis =1)

to_ohe = ['Country', 'Sector', 'Market-Cap']

model_df = one_h_e(model_df, to_ohe)

# Scale for subsequent iterations
scaler = StandardScaler()
model_df = pd.DataFrame(scaler.fit_transform(model_df), columns = model_df.columns)

In [492]:
model_df

MarketCapitalization    EBITDA   PERatio  PEGRatio  BookValue  \
0                 0.202591 -0.058414 -0.024058 -0.024832  -0.060709   
1                -0.135087 -0.058239 -0.098233 -0.053461  -0.060377   
2                -0.210617 -0.060266 -0.132861 -0.036136  -0.060205   
3                -0.185837 -0.061803 -0.021840 -0.020078  -0.061487   
4                -0.095266 -0.059356 -0.093058 -0.039675  -0.057590   
...                    ...       ...       ...       ...        ...   
1899              0.473196 -0.059664  0.059357 -0.022053  -0.060765   
1900              0.018738 -0.046775 -0.049702  0.202724  -0.056996   
1901              0.642586 -0.055258 -0.017348 -0.020796  -0.061327   
1902             -0.209808 -0.061651 -0.135050 -0.041735  -0.060111   
1903             -0.215332 -0.062045  0.053131 -0.050398  -0.061977   

      DividendYield       EPS  RevenuePerShareTTM  ProfitMargin  \
0         -0.275118 -0.033138           -0.066475     -0.045184   
1         -0.235492 -0.033548           -0.061795     -0.112135   
2         -0.243983 -0.033209           -0.062142     -0.102970   
3         -0.268042 -0.033942           -0.067807     -0.051890   
4         -0.229831 -0.030393           -0.047989     -0.103137   
...             ...       ...                 ...           ...   
1899      -0.345880 -0.033091           -0.067452      0.019085   
1900      -0.198696 -0.034219           -0.064536     -0.051331   
1901      -0.282195 -0.032785           -0.067055      0.009026   
1902      -0.345880 -0.032430           -0.063138     -0.075921   
1903      -0.221340 -0.034521           -0.068697     -0.106211   

      OperatingMarginTTM  ReturnOnAssetsTTM  ReturnOnEquityTTM  RevenueTTM  \
0               0.040109           0.244064           0.105046   -0.068285   
1              -0.123395          -0.096230           0.105037   -0.066477   
2              -0.085663           0.165389          -9.468744   -0.070095   
3              -0.019982           0.590046           0.105047   -0.070643   
4              -0.139466          -0.061158           0.105044   -0.066299   
...                  ...                ...                ...         ...   
1899            0.137932           0.808062           0.105060   -0.069334   
1900           -0.029764          -0.026086           0.105036   -0.058843   
1901            0.241344           0.627962           0.105076   -0.067754   
1902           -0.067496           0.428904           0.105049   -0.070375   
1903           -0.175241           0.085766           0.105039   -0.070815   

      GrossProfitTTM  DilutedEPSTTM  QuarterlyEarningsGrowthYOY  \
0          -0.054822      -0.033138                   -0.070383   
1          -0.055546      -0.033548                   -0.060478   
2          -0.055683      -0.033209                   -0.069234   
3          -0.056138      -0.033942                   -0.073116   
4          -0.054014      -0.030393                   -0.074543   
...              ...            ...                         ...   
1899       -0.055317      -0.033091                   -0.066585   
1900       -0.053350      -0.034219                   -0.076429   
1901       -0.053939      -0.032785                   -0.070262   
1902       -0.056041      -0.032430                   -0.075377   
1903       -0.056184      -0.034521                   -0.075885   

      QuarterlyRevenueGrowthYOY  AnalystTargetPrice  TrailingPE  ForwardPE  \
0                     -0.023852           -0.016138   -0.024058   0.222671   
1                     -0.023722           -0.026910   -0.098233  -0.162672   
2                     -0.024221           -0.028569   -0.132861  -0.133654   
3                     -0.024091           -0.026310   -0.021840  -0.292160   
4                     -0.024276           -0.011230   -0.093058  -0.034276   
...                         ...                 ...         ...        ...   
1899                  -0.023251            0.001962    0.059

In [472]:
# Determining the number of clusters
k_means_3 = KMeans(n_clusters=3).fit(model_df)
k_means_4 = KMeans(n_clusters=4).fit(model_df)
k_means_5 = KMeans(n_clusters=5).fit(model_df)
k_means_6 = KMeans(n_clusters=6).fit(model_df)
k_means_7 = KMeans(n_clusters=7).fit(model_df)
k_means_8 = KMeans(n_clusters=8).fit(model_df)
k_means_9 = KMeans(n_clusters=9).fit(model_df)
k_means_10 = KMeans(n_clusters=10).fit(model_df)
k_means_11 = KMeans(n_clusters=11).fit(model_df)
k_means_12 = KMeans(n_clusters=12).fit(model_df)
k_means_13 = KMeans(n_clusters=13).fit(model_df)
k_means_14 = KMeans(n_clusters=14).fit(model_df)

k_list = [k_means_3, k_means_4,k_means_5, k_means_6, k_means_7, k_means_8, 
          k_means_9, k_means_10, k_means_11, k_means_12, k_means_13, k_means_14]

In [473]:
# Calculate Calinski-Harabasz score for different n of clusters
CH_score = []
for i in k_list:
    label = i.labels_
    CH_score.append(calinski_harabasz_score(model_df, label))

# Plot CH elbow graph
plt.plot(list(range(3,15)), CH_score)
plt.xticks(list(range(3,15)))
plt.title('Calinski Harabasz Scores for Different Values of K')
plt.ylabel('Variance Ratio')
plt.xlabel('K=')
plt.show()

In [474]:
# Calculate the Within Cluster Sum of Squares for different n clusters
wcss_score = []
for i in k_list:
    wcss_score.append(i.inertia_)

# Plot WCSS elbow graph
plt.plot(list(range(3,15)), wcss_score)
plt.xticks(list(range(3,15)))
plt.title('Within Cluster Sum of Squares')
plt.ylabel('WCSS')
plt.xlabel('K=')
plt.show()

In [475]:
cluster7_preds = k_means_7.predict(model_df)

In [476]:
testtest = model_df.copy()

In [477]:
testtest['preds'] = cluster7_preds

In [478]:
testtest['preds'].value_counts()

2    1273
1     550
0      65
5       8
6       5
3       2
4       1
Name: preds, dtype: int64

In [102]:
testtest[['Symbol', 'Name']] = model_stocks[['Symbol', 'Name']]

In [106]:
testtest

MarketCapitalization    EBITDA   PERatio  PEGRatio  BookValue  \
0                 0.402336 -0.080865  0.046726 -0.035930  -0.065570   
1                -0.283513 -0.080738 -0.128404 -0.056784  -0.064868   
2                -0.436918 -0.082210 -0.210163 -0.044164  -0.064503   
3                -0.386588 -0.083326  0.051962 -0.032467  -0.067220   
4                -0.202633 -0.081549 -0.116187 -0.046742  -0.058961   
...                    ...       ...       ...       ...        ...   
996               0.218155 -0.079804 -0.082557 -0.042332  -0.073373   
997               0.010330 -0.081013 -0.102359 -0.038562  -0.065700   
998               0.105646 -0.079657 -0.051076 -0.044418  -0.057528   
999              -0.448430 -0.083373 -0.232784 -0.051013  -0.062713   
1000              1.295995 -0.078572  0.062568 -0.032990  -0.066881   

      DividendPerShare  DividendYield       EPS  RevenuePerShareTTM  \
0            -0.050555      -0.335089 -0.076497           -0.065891   
1            -0.051204      -0.299704 -0.115980           -0.060804   
2            -0.051463      -0.307287 -0.083347           -0.061181   
3            -0.051172      -0.328771 -0.154066           -0.067338   
4            -0.048959      -0.294649  0.188407           -0.045799   
...                ...            ...       ...                 ...   
996          -0.048604      -0.196077 -0.026762           -0.065775   
997          -0.051204      -0.321188 -0.113624           -0.065431   
998          -0.050235      -0.316133 -0.024930           -0.063376   
999          -0.051657      -0.357837 -0.019259           -0.048475   
1000         -0.050332      -0.341408 -0.042425           -0.066521   

      ProfitMargin  OperatingMarginTTM  ReturnOnAssetsTTM  ReturnOnEquityTTM  \
0         0.051406            0.161742           0.373071           0.122690   
1        -0.474084           -0.408377          -0.215306           0.122682   
2        -0.402147           -0.276811           0.237039          -8.107101   
3        -0.001231           -0.047789           0.971281           0.122691   
4        -0.403463           -0.464414          -0.154665           0.122688   
...            ...                 ...                ...                ...   
996       0.275112            0.761097           2.642991          -8.107101   
997      -0.198619           -0.359649           0.205900           0.122690   
998      -0.145982           -0.072153          -0.420172           0.122680   
999      -0.516632           -0.667123           0.061674           0.122686   
1000      0.476886            0.863426           1.036838           0.122716   

      RevenueTTM  GrossProfitTTM  DilutedEPSTTM  QuarterlyEarningsGrowthYOY  \
0      -0.081201       -0.084703      -0.076497                   -0.073646   
1      -0.079862       -0.087148      -0.115980                   -0.066307   
2      -0.082541       -0.087610      -0.083347                   -0.072794   
3      -0.082947       -0.089148      -0.154066                   -0.075671   
4      -0.079730       -0.081971       0.188407                   -0.076728   
...          ...             ...            ...                         ...   
996    -0.081159       -0.084943      -0.026762                   -0.068440   
997    -0.080124       -0.086697      -0.113624                   -0.074622   
998    -0.080660       -0.081289      -0.024930                   -0.076378   
999    -0.082594       -0.088983      -0.019259                    0.256582   
1000   -0.080807       -0.081719      -0.042425                   -0.073556   

      QuarterlyRevenueGrowthYOY  AnalystTargetPrice  TrailingPE  ForwardPE  \
0                     -0.179462            0.586224    0.046726   0.260488   
1                     -0.078445           -0.322560   -0.128404  -0.179809   
2                     -0.465954           -0.462544   -0.210163  -0.146653   
3                     -0.364937           -0.271947    0.051962  -0.327763   


In [103]:
testtest[testtest['preds']== 6]

MarketCapitalization    EBITDA   PERatio  PEGRatio  BookValue  \
7                3.178510 -0.039093 -0.071146 -0.044710  -0.067258   
8                0.361989 -0.050050 -0.160758 -0.040124  -0.067642   
10               3.433157 -0.064807 -0.048391 -0.033867  -0.065069   
76               0.611915  0.385058 -0.192374 -0.048435  -0.054543   
95              -0.176743  0.068818 -0.191972 -0.043933  -0.048263   
112              2.087323 -0.025761 -0.189757 -0.054553  -0.064849   
136              2.189897 -0.025761 -0.186736 -0.054553  -0.064849   
143              1.369749 -0.051081 -0.194254 -0.052629  -0.064291   
154              1.031500 -0.064125 -0.207411  0.028558  -0.063526   
189              4.026719 -0.032371 -0.139681 -0.042301  -0.064774   
211              3.886315 -0.058638 -0.122564 -0.032782  -0.066767   
218              0.911151 -0.073470 -0.122497 -0.042579  -0.067468   
234              1.617927 -0.055195 -0.167337 -0.040716  -0.058388   
235              3.449877 -0.048411  0.115194 -0.050466  -0.055954   
251              5.430084 -0.071223  1.641894 -0.046519  -0.059730   
277             -0.168264 -0.075669 -0.109609 -0.025387  -0.067210   
283              1.106252 -0.063756 -0.155388 -0.016537  -0.063656   
293              0.502927 -0.070911 -0.180493  0.091352  -0.066484   
297              0.046349 -0.063319 -0.221977 -0.053683  -0.067097   
310              0.674751 -0.066541 -0.151965 -0.053483  -0.066971   
317              0.509695 -0.070905 -0.147468 -0.053014  -0.067062   
362              1.136326 -0.060646 -0.159013 -0.050505  -0.065684   
371              1.122026 -0.051044 -0.225467 -0.049250  -0.062254   
382              1.338390 -0.066345 -0.162302 -0.028665  -0.067435   
400              0.526009  3.494279 -0.229696  0.012090   0.919882   
411             -0.091649 -0.075858 -0.143373 -0.048165  -0.066176   
412              0.140530 -0.073799 -0.212713 -0.047565  -0.068699   
434              1.492614 -0.057841 -0.108870 -0.043094  -0.064079   
451              1.333539 -0.077286 -0.045571 -0.033913  -0.068142   
481              7.403012 -0.034294 -0.109206 -0.042086  -0.063720   
493              0.384451 -0.072359 -0.129679 -0.026310  -0.061077   
500              0.295469 -0.072976 -0.112294 -0.039939  -0.066077   
506              3.906849 -0.062645 -0.076382 -0.035776  -0.067615   
509              0.094546 -0.074570 -0.093230 -0.024848  -0.066277   
522              1.661806  0.053626 -0.235805 -0.052190  -0.053295   
558              1.093725 -0.073623 -0.138876 -0.027796  -0.064957   
560              2.654479 -0.068818  0.020212 -0.036630  -0.061551   
567              0.261863 -0.067695 -0.225400 -0.040393  -0.063921   
578              1.150810 -0.063879 -0.142702 -0.024879  -0.068236   
588              3.320048 -0.051558 -0.026306 -0.043340  -0.066154   
603              0.997829 -0.062629 -0.180493 -0.048096  -0.061374   
610              2.390428 -0.071162  0.055587 -0.029219  -0.065113   
624              3.977186 -0.070131 -0.003417 -0.032120  -0.066901   
644              9.266206 -0.069890  0.232529 -0.030912  -0.067009   
646              3.742053  0.010819 -0.048860 -0.026687  -0.063345   
647              2.977005 -0.052911 -0.130888 -0.037838  -0.064137   
669              0.086724 -0.062921 -0.065172 -0.039547  -0.066500   
670              4.414210 -0.054430 -0.134982 -0.035476  -0.068219   
689              3.939021 -0.051187 -0.150354 -0.040455  -0.066270   
691              5.983998 -0.048953 -0.097191 -0.026326  -0.065131   
707              2.304647 -0.061338 -0.158409 -0.043633  -0.069877   
732              2.171525 -0.064523 -0.166128 -0.049689  -0.067232   
741              2.952017  0.002731 -0.049397 -0.043548  -0.060665   
742              2.954276  0.002731 -0.049733 -0.043725  -0.060665   
757              1.651755 -0.032172 -0.230233 -0.054553  -0.062567   
814             -0.004043 -0.079502  0.407189 -0.051290  -0.068579   
822            

In [104]:
testtest[testtest['preds']== 4]

MarketCapitalization    EBITDA    PERatio  PEGRatio  BookValue  \
1               -0.283513 -0.080738  -0.128404 -0.056784  -0.064868   
31              -0.378449 -0.082435  -0.162571  0.239951  -0.067174   
35              -0.399024 -0.083233  -0.056848 -0.054553  -0.067556   
45              -0.431785 -0.083443  -0.142030 -0.036037  -0.063405   
56              -0.388881 -0.083395   0.228099 -0.165905  -0.056712   
77              -0.358428 -0.081727  -0.231710 -0.054553  -0.022511   
79              -0.412787 -0.082764  -0.213183 -0.243244  -0.065627   
90              -0.348249 -0.082748  -0.060003 -0.065018  -0.058910   
119             -0.424941 -0.083312  -0.065507 -0.033390  -0.060339   
120             -0.395109 -0.082651   0.289519 -0.032775  -0.067920   
123             -0.409319 -0.083189  -0.217345  0.060417  -0.066696   
132             -0.444014 -0.083261  -0.238631 -0.020308  -0.066372   
133             -0.418385 -0.082954  -0.154717  0.006472  -0.066184   
137             -0.426444 -0.082732  -0.228085 -0.047781  -0.062075   
140             -0.413324 -0.082922  -0.136795 -0.046318  -0.068256   
142             -0.439616 -0.083498   1.711503 -0.045272  -0.065177   
156             -0.422212 -0.083473   3.198598 -0.048781  -0.064829   
164             -0.392320 -0.083102  -0.183044 -0.035314  -0.063391   
170             -0.415366 -0.082737  -0.232515 -0.051090  -0.060490   
181             -0.427380 -0.083407   0.012157 -0.049951  -0.067967   
182             -0.251631 -0.079627  -0.187340 -0.056246  -0.067375   
183             -0.084897 -0.078907   0.075791 -0.035991  -0.065225   
198             -0.380130 -0.083239   0.256292 -0.013844  -0.063887   
222             -0.125737 -0.082271  -0.057318 -0.051944  -0.067507   
224             -0.441404 -0.083343  -0.200899 -0.046242  -0.065644   
239             -0.387843 -0.082202  -0.223118 -0.041470  -0.067887   
242             -0.346419 -0.082684   0.536339  0.015399  -0.066033   
253             -0.262719 -0.080133  -0.207209  0.017400  -0.062023   
257             -0.424825 -0.083012  -0.160557 -0.040393  -0.066184   
279             -0.359952 -0.082993  -0.209491 -0.049427  -0.064879   
286             -0.404519 -0.082507   1.568593 -0.043009  -0.067647   
292             -0.418255 -0.082967  -0.132566 -0.036161  -0.063469   
352             -0.434255 -0.082868  -0.219762 -0.049420  -0.067051   
353             -0.429299 -0.083071  -0.219292 -0.007303  -0.066841   
374             -0.390788 -0.082158  -0.226340 -0.049643  -0.050167   
393             -0.432771 -0.083491   0.002356 -0.041470  -0.063762   
396             -0.420197 -0.083050  -0.215667 -0.032928  -0.067920   
408             -0.346805 -0.082063  -0.191905 -0.052598  -0.065960   
414             -0.370345 -0.081984  -0.212512 -0.049004  -0.068169   
421             -0.321155 -0.082792   0.319054 -0.054553  -0.065966   
423              0.132401 -0.081128   0.802021  0.301437  -0.061806   
429             -0.442114 -0.083321  -0.220970 -0.042933  -0.065673   
440             -0.415032 -0.083321   0.207089 -0.042602  -0.064381   
459             -0.223437 -0.081179  -0.134311 -0.046242  -0.067834   
461             -0.413288 -0.083338   0.458809 -0.019385  -0.067262   
478             -0.399095 -0.083351   0.185810 -0.014152  -0.060541   
483             -0.418961 -0.082837   0.040081 -0.044241  -0.066253   
511             -0.418931 -0.083276   0.691935 -0.054553  -0.065933   
513             -0.329114 -0.079711  -0.207478  0.101510  -0.062274   
517             -0.376632 -0.083077  -0.082692 -0.034006  -0.054484   
529             -0.432064 -0.083437  -0.012613 -0.027834  -0.066914   
535             -0.178309 -0.081340  -0.176197 -0.039854  -0.065986   
549             -0.330999 -0.082127  -0.133304 -0.050320  -0.068283   
573             -0.425500 -0.083356  -0.123168 -0.035545  -0.063612   
577             -0.419052 -0.083239  -0.001806 -0.040778  -0.064416   
598             -0.352880 -0.

In [105]:
testtest[testtest['preds']== 1]

MarketCapitalization    EBITDA   PERatio  PEGRatio  BookValue  \
2               -0.436918 -0.082210 -0.210163 -0.044164  -0.064503   
3               -0.386588 -0.083326  0.051962 -0.032467  -0.067220   
5               -0.410023 -0.083212  0.591449  0.113592  -0.064772   
6                0.781260 -0.077702 -0.190294 -0.052452  -0.067287   
9               -0.395259 -0.082765 -0.124309 -0.020308  -0.064258   
..                    ...       ...       ...       ...        ...   
994             -0.384905 -0.082177 -0.173042 -0.051598  -0.063248   
995             -0.055065 -0.082111  0.105192 -0.035453  -0.065470   
997              0.010330 -0.081013 -0.102359 -0.038562  -0.065700   
998              0.105646 -0.079657 -0.051076 -0.044418  -0.057528   
999             -0.448430 -0.083373 -0.232784 -0.051013  -0.062713   

     DividendPerShare  DividendYield       EPS  RevenuePerShareTTM  \
2           -0.051463      -0.307287 -0.083347           -0.061181   
3           -0.051172      -0.328771 -0.154066           -0.067338   
5           -0.050074      -0.048218 -0.203278           -0.067917   
6           -0.050326      -0.054537 -0.096348           -0.066786   
9           -0.050566      -0.192286 -0.123222           -0.056120   
..                ...            ...       ...                 ...   
994         -0.050171       0.202004 -0.154503           -0.063607   
995         -0.050041      -0.285803 -0.123964           -0.064655   
997         -0.051204      -0.321188 -0.113624           -0.065431   
998         -0.050235      -0.316133 -0.024930           -0.063376   
999         -0.051657      -0.357837 -0.019259           -0.048475   

     ProfitMargin  OperatingMarginTTM  ReturnOnAssetsTTM  ReturnOnEquityTTM  \
2       -0.402147           -0.276811           0.237039          -8.107101   
3       -0.001231           -0.047789           0.971281           0.122691   
5       -0.420131            0.293308          -0.705347           0.122675   
6        0.235634           -0.421046          -0.339865           0.122679   
9       -0.549091           -0.594031           0.028895           0.122681   
..            ...                 ...                ...                ...   
994     -0.473645           -0.541892          -0.610288           0.122678   
995     -0.330210           -0.315794          -0.251362           0.122684   
997     -0.198619           -0.359649           0.205900           0.122690   
998     -0.145982           -0.072153          -0.420172           0.122680   
999     -0.516632           -0.667123           0.061674           0.122686   

     RevenueTTM  GrossProfitTTM  DilutedEPSTTM  QuarterlyEarningsGrowthYOY  \
2     -0.082541       -0.087610      -0.083347                   -0.072794   
3     -0.082947       -0.089148      -0.154066                   -0.075671   
5     -0.083000       -0.089032      -0.203278                   -0.075267   
6     -0.074389       -0.076126      -0.096348                   -0.063745   
9     -0.081240       -0.087916      -0.123222                   -0.061738   
..          ...             ...            ...                         ...   
994   -0.080873       -0.085053      -0.154503                   -0.048217   
995   -0.081493       -0.086362      -0.123964                   -0.053011   
997   -0.080124       -0.086697      -0.113624                   -0.074622   
998   -0.080660       -0.081289      -0.024930                   -0.076378   
999   -0.082594       -0.088983      -0.019259                    0.256582   

     QuarterlyRevenueGrowthYOY  AnalystTargetPrice  TrailingPE  ForwardPE  \
2                    -0.465954           -0.462544   -0.210163  -0.146653   
3                    -0.364937           -0.271947    0.051962  -0.327763   
5                    -0.412961           -0.437425    0.591449  -0.327763   
6                    -0.258951           -0.445475   -0.190294   0.024474   
9                    -0.429521           -0.271947  

# Finding All stocks on NYSE and NASDAQ

In [42]:
NYSE = pd.read_csv('data/NYSE_LIST_OF_SYMBOLS.csv')
NASDAQ = pd.read_csv('data/NASDAQ_LIST_OF_SYMBOLS.csv')

stock_list = pd.concat([NYSE, NASDAQ])

In [43]:
stock_list.head()

Code                                               Name Country   Exchange  \
0     A                           Agilent Technologies Inc     USA       NYSE   
1    AA                                  Alcoa Corporation     USA       NYSE   
2   AAA  Listed Funds Trust - AAF First Priority CLO Bo...     USA  NYSE ARCA   
3  AAAU                    Goldman Sachs Physical Gold ETF     USA  NYSE ARCA   
4   AAC                       Ares Acquisition Corporation     USA       NYSE   

  Currency          Type          Isin  
0      USD  Common Stock  US00846U1016  
1      USD  Common Stock  US0138721065  
2      USD           ETF           NaN  
3      USD           ETF  US7154261025  
4      USD  Common Stock  KYG330321061

In [44]:
stock_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11005 entries, 0 to 5263
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Code      11005 non-null  object
 1   Name      11003 non-null  object
 2   Country   11003 non-null  object
 3   Exchange  11003 non-null  object
 4   Currency  11003 non-null  object
 5   Type      11003 non-null  object
 6   Isin      8539 non-null   object
dtypes: object(7)
memory usage: 687.8+ KB


In [45]:
stock_list.Type.value_counts()

Common Stock       8029
ETF                2171
FUND                442
Preferred Share     328
Mutual Fund          22
Fund                  8
Preferred Stock       2
BOND                  1
Name: Type, dtype: int64

In [46]:
stocks = stock_list[stock_list['Type'] == 'Common Stock'].sort_values(by='Code')
stocks.shape

(8029, 7)

In [47]:
stocks.to_csv('data/stock_tickers')

# Downloading Stock Info


In [ ]:
key_alpha_old = 'C9TOSUV8PMESV234'
key_alpha = 'YTEP4AMCUANDLBKB'
key_EOD = '6164aa3912fe92.68825657'

In [ ]:
# base_data = pd.DataFrame(columns = ['Symbol', 'AssetType', 'Name', 'Description', 'CIK', 'Exchange', 
#                                     'Currency', 'Country', 'Sector', 'Industry', 'Address', 'FiscalYearEnd', 
#                                     'LatestQuarter', 'MarketCapitalization', 'EBITDA', 'PERatio', 'PEGRatio', 
#                                     'BookValue', 'DividendPerShare', 'DividendYield', 'EPS', 'RevenuePerShareTTM', 
#                                     'ProfitMargin', 'OperatingMarginTTM', 'ReturnOnAssetsTTM', 'ReturnOnEquityTTM', 
#                                     'RevenueTTM', 'GrossProfitTTM', 'DilutedEPSTTM', 'QuarterlyEarningsGrowthYOY', 
#                                     'QuarterlyRevenueGrowthYOY', 'AnalystTargetPrice', 'TrailingPE', 'ForwardPE', 
#                                     'PriceToSalesRatioTTM', 'PriceToBookRatio', 'EVToRevenue', 'EVToEBITDA', 'Beta', 
#                                     '52WeekHigh', '52WeekLow', '50DayMovingAverage', '200DayMovingAverage', 
#                                     'SharesOutstanding', 'SharesFloat', 'SharesShort', 'SharesShortPriorMonth', 
#                                     'ShortRatio', 'ShortPercentOutstanding', 'ShortPercentFloat', 'PercentInsiders', 
#                                     'PercentInstitutions', 'ForwardAnnualDividendRate', 'ForwardAnnualDividendYield', 
#                                     'PayoutRatio', 'DividendDate', 'ExDividendDate', 'LastSplitFactor', 'LastSplitDate'])


In [ ]:
# %%time
# for i in stocks.Code.values[160:]:
#     url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={i}&apikey={key_alpha}'
#     r = requests.get(url)
#     data_overview = r.json()
    
#     if data_overview == {}:
#         continue
#     else:
#         base_data = base_data.append(data_overview, ignore_index = True)

In [ ]:
# base_data.to_csv('data/base_data.csv')

# Cleaning Base Stock info

In [159]:
base_1 = pd.read_csv('data/base_data.csv')

In [169]:
base_1['state'] = base_1['Address'].map(lambda x: str(x)[-6:])

In [177]:
base_1['Industry'].value_counts()

BLANK CHECKS                                   975
GENERAL                                        683
PHARMACEUTICAL PREPARATIONS                    455
REAL ESTATE INVESTMENT TRUSTS                  196
STATE COMMERCIAL BANKS                         188
                                              ... 
STRUCTURAL CLAY PRODUCTS                         1
CUT STONE & STONE PRODUCTS                       1
PUBLIC BLDG & RELATED FURNITURE                  1
SERVICES-NURSING & PERSONAL CARE FACILITIES      1
ELECTRIC, GAS & SANITARY SERVICES                1
Name: Industry, Length: 375, dtype: int64

**Justification for dropping columns:**

1. **AssetType** - Since we are only looking at Common Stock Assets we can drop the column
2. **Currency** - All Currencies were USD
3. **Address** - The Headquarters of a company may have some bearing what kind of company it is (think Silicon Valley) but we want to focus more on stock features and performance.
4. **CIK** - Central Index key, is just a unique id for each stock. 
5. **FiscalYearEnd** - The month in which the company registers its year end. I don't think it's a relevant feature for seperating stocks.
6. **LatestQuarter** - 
7. **Industry** - The industry segment was split into too many niche categories (375) and the majority of stocks were saved as "Blank Checks" or "General"
8. **Exchange** - After reading a bit about the difference between the NASDAQ and NYSE and breaking down the .describe of each exchange the key differences seem to be that the biggest stocks (Apple, Microsoft, Alphabet, Amazon, Facebook) are all on the NASDAQ, but also a lot of the smallest stocks are on NASDAQ. May return this column in subsequent iterations, but for now, we are dropping it. 
9. **LastSplitDate** - Turned into two boolean columns: one if the stock has ever split, and two if the stock has split in the last five years. This hopes to seperate out stocks that grew in price far enough that it made sense to split the stock to keep the individual share price at a reasonable amount.
10. **LastSplitFactor** - Removed in favour of Last Split Date. There may be some favour in adding it for showing how high the ratio of a stock split was, but because adding it may skew the data in favour of stocks that have had splits, it was removed. 
11. **Dividend Date** - 
12. **ExDividendDate** - 
13. **DividendPerShare** - Too many null values to try and salvage


**Justification for filling null variables:**

1. **Beta** - Because Beta is a pretty important feature and a lot of cool stocks did not have beta data we filled the remaining null values with normally distributed datapoints with the same mean and std. 

In [393]:
def Pearson_Corr_matrix(cleaned_df):
    '''Use this function to print out a Pearson Correlation Matrix for a cleaned dataframe.
    The dataframe should not have any null values'''
    features = [i for i in cleaned_df.columns]

    mask = np.zeros_like(cleaned_df[features].corr(), dtype=np.bool) 
    mask[np.triu_indices_from(mask)] = True 

    f, ax = plt.subplots(figsize=(16, 12))
    plt.title('Pearson Correlation Matrix',fontsize=25)

    sns.heatmap(cleaned_df[features].corr(),annot=True, cmap='BuGn', 
                linecolor="w", mask = mask)
    
def print_regplot(cleaned_df, x, y):
    '''Print a regplot for a cleaned dataframe.
    Inputs:
    cleaned_df - try to remove empty rows
    x - the x value you are using to predict y - for selecting x value run Pearson_Corr_matrix
    y - the output value you are trying to fill null values for
    '''
    # Plot using table above
    plt.figure(figsize =(12,8))
    sns.regplot(data= cleaned_df, x= x, y= y,
               marker= '+', x_estimator=np.mean,
               line_kws={"color": "orange"})
    plt.ylabel(y, fontsize = 14)
    plt.xlabel(x, fontsize = 14)
    plt.title(y, fontsize = 17, fontweight = 'bold')
    plt.show()

    the_coeff = np.corrcoef(cleaned_df[x], cleaned_df[y])[0][1]

    #Just in case the visualization is not clear, let's calculate it.
    print("Correlation Coefficient:\n", np.corrcoef(cleaned_df[x], cleaned_df[y]))
    
    return the_coeff

In [451]:
base_df = pd.read_csv('data/base_data.csv')

In [452]:
## Cleaning base_data ##

## Drop all null values by Symbol
base_df.dropna(subset = ['Symbol'], inplace=True)

## Replace all 'None' with Null
for i in ['PERatio', 'PEGRatio', 'BookValue', 'DividendPerShare', 'DividendYield', 'EPS', 
          'MarketCapitalization', 'EBITDA', 'Beta', 'AnalystTargetPrice']:
    base_df[i] = base_df[i].replace('None', np.NaN)
    

## Change to columns to float or int
for i in ['PERatio', 'PEGRatio', 'BookValue', 'DividendPerShare', 'DividendYield', 'EPS', 
          'MarketCapitalization', 'EBITDA', 'Beta', 'AnalystTargetPrice']:
    base_df[i] = base_df[i].map(lambda x: float(x))
    
## Drop Unnamed: 0
base_df.drop(['Unnamed: 0', 'Information', 'Error Message'], axis = 1, inplace=True)

## Change index name so it's easier to save and re-load
base_df.index.names = ['Index']

## Bin Market Cap
bins = [0, 300000000, 2000000000, 10000000000, 9999999999999]
labels = ['Micro-Cap', 'Small-Cap', 'Mid-Cap', 'Big-Cap']

base_df['Market-Cap'] = pd.cut(base_df['MarketCapitalization'], bins=bins, labels=labels)

## Used to check NASDAQ vs NYSE
# base_df[base_df['Exchange'] == 'NYSE'].describe()
# base_df[base_df['Exchange'] == 'NASDAQ'].describe()

## Add columns split_ever and split_last_5_yrs
base_df['split_ever'] = base_df['LastSplitDate'].map(lambda x: 0 if (x == 'None') else 1)

split_last_5_yrs = []
for i in base_df['LastSplitDate']:
    try:
        if int(i[:4]) > 2016:
            split_last_5_yrs.append(1)
        else:
            split_last_5_yrs.append(0)
    except:
        split_last_5_yrs.append(0)
base_df['split_last_5_yrs'] = split_last_5_yrs

## Add columns dividend_ever and dividend_last_3_yrs
base_df['dividend_ever'] = base_df['DividendDate'].map(lambda x: 0 if (x == 'None') else 1)

dividend_last_3_yrs = []
for i in base_df['DividendDate']:
    try:
        if int(i[:4]) > 2018:
            dividend_last_3_yrs.append(1)
        else:
            dividend_last_3_yrs.append(0)
    except:
        dividend_last_3_yrs.append(0)
base_df['dividend_3_yrs'] = dividend_last_3_yrs


## Fill EPS Null values using multicollinear features
# Create new df with only EPS not null values 
EPS = base_df[base_df['EPS'].notna()]

# Drop Berkshire Hathway and LILM because of the significant outliers
EPS = EPS[EPS['EPS'] != 68287.15]
EPS = EPS[EPS['Symbol'] != 'LILM']

# Get rid of columns with most missing values
EPS.drop(['PERatio', 'Beta', 'Market-Cap', 'DividendPerShare', 'DividendYield', 'PEGRatio',
          'EBITDA', 'AnalystTargetPrice'], axis=1, inplace = True)

# Remove object strings
EPS.drop(['Symbol', 'Name', 'Description', 'Country', 'Sector'], axis = 1, inplace=True)

# Remove remaining null rows (only about a hundred)
EPS.dropna(inplace=True)

# Pearson_Corr_matrix(EPS)

EPS_coeff = print_regplot(EPS, 'RevenuePerShareTTM', 'EPS')

EPS_to_add = []
for n,i in enumerate(base_df['EPS']):
    if np.isnan(i):
        EPS_to_add.append(base_df['RevenuePerShareTTM'].iloc[n] * EPS_coeff)
    else: 
        EPS_to_add.append(i)

base_df['EPS'] = EPS_to_add

## Fill beta missing values
Beta = base_df[base_df['Beta'].notna()]

for i in ['EBITDA', 'PERatio', 'PEGRatio', 'BookValue', 'DividendYield']:
    Beta[i] = Beta[i].fillna(Beta[i].mean())
    
Beta.drop(['Symbol', 'Name', 'Description', 'Country', 'Sector', 
           'Market-Cap', 'AnalystTargetPrice'], axis = 1, inplace=True)

# No promising multicollinearity, will resort to filling with normally distributed data
# Pearson_Corr_matrix(Beta)

# Fillna w/normally distributed features
index = base_df[base_df.Beta.isna()].index
value = np.random.normal(loc=base_df.Beta.mean(), scale=base_df.Beta.std(), size=base_df.Beta.isna().sum())

base_df.Beta.fillna(pd.Series(value, index=index), inplace=True)

## Fill EBITDA Missing Values
EBITDA = base_df[base_df['EBITDA'].notna()]

for i in ['PERatio', 'PEGRatio', 'BookValue', 'DividendYield']:
    EBITDA[i] = EBITDA[i].fillna(EBITDA[i].mean())
    
EBITDA.drop(['Symbol', 'Name', 'Description', 'Country', 'Sector', 
           'Market-Cap', 'AnalystTargetPrice'], axis = 1, inplace=True)

# Multicollinearity shows v high correlation with RevenuePerShareTT
EBITDA_coeff = print_regplot(EBITDA, 'RevenuePerShareTTM', 'EBITDA')

EBITDA_to_add = []
for n,i in enumerate(base_df['EBITDA']):
    if np.isnan(i):
        EBITDA_to_add.append(base_df['RevenuePerShareTTM'].iloc[n] * EBITDA_coeff)
    else: 
        EBITDA_to_add.append(i)

base_df['EBITDA'] = EBITDA_to_add

## Drop Asset Type
base_df.drop(['AssetType', 'Currency', 'Address', 'CIK', 'FiscalYearEnd', 'LatestQuarter', 
              'Industry', 'Exchange', 'LastSplitDate', 'LastSplitFactor', 'ExDividendDate',
              'DividendDate', 'ForwardAnnualDividendRate', 'ForwardAnnualDividendYield',
              'DividendPerShare'], axis=1, inplace = True)

## Remove null values 
base_df = base_df[base_df['MarketCapitalization'].notna()]

## Reset Index to not skip rows that we did not pull data for 
base_df.reset_index(drop = True, inplace=True)

base_df.to_csv('data/clean_df.csv')

PERatio
PEGRatio
BookValue
DividendPerShare
DividendYield
EPS
MarketCapitalization
EBITDA
Beta
AnalystTargetPrice


Correlation Coefficient:
 [[1.         0.04362026]
 [0.04362026 1.        ]]


Correlation Coefficient:
 [[1.        0.4960739]
 [0.4960739 1.       ]]


In [453]:
base_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5253 entries, 0 to 5252
Data columns (total 49 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   Symbol                      5253 non-null   object  
 1   Name                        5253 non-null   object  
 2   Description                 5253 non-null   object  
 3   Country                     5253 non-null   object  
 4   Sector                      5253 non-null   object  
 5   MarketCapitalization        5253 non-null   float64 
 6   EBITDA                      5253 non-null   float64 
 7   PERatio                     2448 non-null   float64 
 8   PEGRatio                    3088 non-null   float64 
 9   BookValue                   4786 non-null   float64 
 10  DividendYield               3689 non-null   float64 
 11  EPS                         5253 non-null   float64 
 12  RevenuePerShareTTM          5253 non-null   float64 
 13  ProfitMargin      

In [493]:
base_df.dropna()

Symbol                           Name  \
0         A       Agilent Technologies Inc   
1        AA              Alcoa Corporation   
9       AAN       Aarons Company Inc (The)   
11     AAON                       AAON Inc   
12      AAP         Advance Auto Parts Inc   
...     ...                            ...   
5227     ZM  Zoom Video Communications Inc   
5239    ZTO       ZTO Express (Cayman) Inc   
5240    ZTS                     Zoetis Inc   
5241   ZUMZ                     Zumiez Inc   
5252   ZYXI                      Zynex Inc   

                                            Description Country  \
0     Agilent Technologies, Inc. is an American anal...     USA   
1     Alcoa Corporation produces and sells bauxite, ...     USA   
9     The Aarons Company Inc. offers purchase and le...     USA   
11    AAON, Inc. engages in the engineering, manufac...     USA   
12    Advance Auto Parts, Inc. (Advance) is an Ameri...     USA   
...                                                 ...     ...   
5227  Zoom Video Communications, Inc. provides a pre...     USA   
5239  ZTO Express (Cayman) Inc. provides express del...   China   
5240  Zoetis Inc. is an American drug company, the w...     USA   
5241  Zumiez Inc., is a specialty retailer of clothi...     USA   
5252  Zynex, Inc. designs, manufactures and markets ...     USA   

                       Sector  MarketCapitalization        EBITDA  PERatio  \
0               LIFE SCIENCES          4.595638e+10  1.651000e+09    47.32   
1               MANUFACTURING          9.137748e+09  1.730000e+09    21.23   
9            TRADE & SERVICES          9.024170e+08  8.132960e+08     9.05   
11                 TECHNOLOGY          3.604326e+09  1.178790e+08    48.10   
12           TRADE & SERVICES          1.347963e+10  1.224954e+09    23.05   
...                       ...                   ...           ...      ...   
5227               TECHNOLOGY          7.546160e+10  1.085708e+09    76.66   
5239  ENERGY & TRANSPORTATION          2.591000e+10  6.916648e+09    38.30   
5240            LIFE SCIENCES          9.393096e+10  3.079000e+09    49.68   
5241         TRADE & SERVICES          9.905820e+08  1.864000e+08     8.28   
5252            LIFE SCIENCES          3.882740e+08  8.569000e+06    74.47   

      PEGRatio  BookValue  DividendYield    EPS  RevenuePerShareTTM  \
0        2.420     16.330         0.0050  3.208              20.070   
1       -0.290     20.170         0.0078  2.303              56.240   
9        1.350     22.160         0.0072  3.051              53.560   
11       2.870      7.320         0.0055  1.430               9.780   
12       1.015     52.440         0.0082  9.280             162.940   
...        ...        ...            ...    ...                 ...   
5227     2.683     15.680         0.0000  3.313              12.520   
5239    23.960     59.320         0.0104  0.818              35.060   
5240     2.802      9.170         0.0045  3.989              15.590   
5241     0.820     23.250         0.0000  4.775              45.860   
5252     0.000      1.648         0.0088  0.150               2.902   

      ProfitMargin  OperatingMarginTTM  ReturnOnAssetsTTM  ReturnOnEquityTTM  \
0           0.1610              0.2180             0.0835             0.2000   
1           0.0412              0.1010             0.0476             0.1050   
9           0.0576              0.1280             0.0752        -99999.9900   
11          0.1490              0.1750             0.1200             0.2160   
12          0.0573              0.0895             0.0513             0.1780   
...            ...                 ...                ...                ...   
5227        0.2760              0.2880             0.1430             0.3430   
5239        0.1500              0.1680             0.0550             0.0977   
5240        0.2580              0.3620             0.1240             0.5190   
5241        0.1060              0.1410             0.1030

In [494]:
base_df.isna()

Symbol   Name  Description  Country  Sector  MarketCapitalization  \
0      False  False        False    False   False                 False   
1      False  False        False    False   False                 False   
2      False  False        False    False   False                 False   
3      False  False        False    False   False                 False   
4      False  False        False    False   False                 False   
...      ...    ...          ...      ...     ...                   ...   
5248   False  False        False    False   False                 False   
5249   False  False        False    False   False                 False   
5250   False  False        False    False   False                 False   
5251   False  False        False    False   False                 False   
5252   False  False        False    False   False                 False   

      EBITDA  PERatio  PEGRatio  BookValue  DividendYield    EPS  \
0      False    False     False      False          False  False   
1      False    False     False      False          False  False   
2      False    False      True      False           True  False   
3      False     True      True      False           True  False   
4      False     True      True       True           True  False   
...      ...      ...       ...        ...            ...    ...   
5248   False    False     False      False          False  False   
5249   False     True      True      False           True  False   
5250   False     True     False      False          False  False   
5251   False     True     False      False          False  False   
5252   False    False     False      False          False  False   

      RevenuePerShareTTM  ProfitMargin  OperatingMarginTTM  ReturnOnAssetsTTM  \
0                  False         False               False              False   
1                  False         False               False              False   
2                  False         False               False              False   
3                  False         False               False              False   
4                  False         False               False              False   
...                  ...           ...                 ...                ...   
5248               False         False               False              False   
5249               False         False               False              False   
5250               False         False               False              False   
5251               False         False               False              False   
5252               False         False               False              False   

      ReturnOnEquityTTM  RevenueTTM  GrossProfitTTM  DilutedEPSTTM  \
0                 False       False           False          False   
1                 False       False           False          False   
2                 False       False           False          False   
3                 False       False           False          False   
4                 False       False           False          False   
...                 ...         ...             ...            ...   
5248              False       False           False          False   
5249              False       False           False          False   
5250              False       False           False          False   
5251              False       False           False          False   
5252              False       False           False          False   

      QuarterlyEarningsGrowthYOY  QuarterlyRevenueGrowthYOY  \
0                          False                      False   
1                          False                      False   
2                          False                      False   
3                          False                      False   
4                          False                      False   
...                          ...                        ...   
5248                       False 

In [454]:
clean_df = pd.read_csv('data/clean_df.csv')

In [455]:
clean_df.describe()

Unnamed: 0  MarketCapitalization        EBITDA       PERatio  \
count  5253.000000          5.253000e+03  5.253000e+03   2448.000000   
mean   2626.000000          1.033850e+10  1.051712e+10     59.801951   
std    1516.554813          6.696259e+10  2.728279e+11    366.592440   
min       0.000000          1.000000e+03 -7.641000e+09      0.004100   
25%    1313.000000          2.334040e+08  0.000000e+00     10.320000   
50%    2626.000000          7.187030e+08  6.922200e+05     18.820000   
75%    3939.000000          3.789314e+09  1.964350e+08     37.732500   
max    5252.000000          2.380597e+12  1.420572e+13  10130.000000   

          PEGRatio      BookValue  DividendYield           EPS  \
count  3088.000000    4786.000000    3689.000000   5253.000000   
mean      3.215341     294.289500       0.033323     25.897844   
std     105.059986    7311.791821       0.678383   1333.402176   
min   -3426.670000    -128.110000       0.000000  -3206.130000   
25%       0.000000       1.373500       0.000000     -0.597000   
50%       0.845000       6.690000       0.000000      0.000000   
75%       2.149250      19.087500       0.020800      1.751000   
max    3262.790000  311184.380000      39.960000  68287.150000   

       RevenuePerShareTTM  ProfitMargin  OperatingMarginTTM  \
count         5253.000000   5253.000000         5253.000000   
mean           210.611034      0.030515         -230.684091   
std           4668.164959      2.012997        11954.960955   
min             -1.998000     -2.994000      -692588.060000   
25%              0.046000      0.000000           -0.027100   
50%              4.739000      0.000000            0.008000   
75%             20.610000      0.111000            0.165000   
max         207493.100000    115.210000       101465.040000   

       ReturnOnAssetsTTM  ReturnOnEquityTTM    RevenueTTM  GrossProfitTTM  \
count        5253.000000        5253.000000  5.253000e+03    5.253000e+03   
mean           -0.043851      -13305.478625  6.337390e+10    4.234795e+10   
std             0.515104       33968.472522  1.440993e+12    1.228182e+12   
min           -31.190000      -99999.990000 -9.478000e+06   -1.231700e+10   
25%            -0.027100          -0.585000  1.517000e+06    0.000000e+00   
50%             0.000100           0.000000  1.774180e+08    6.719900e+07   
75%             0.039600           0.122000  1.458753e+09    5.393000e+08   
max             2.940000        4081.010000  6.388662e+13    8.207300e+13   

       DilutedEPSTTM  QuarterlyEarningsGrowthYOY  QuarterlyRevenueGrowthYOY  \
count    5253.000000                 5253.000000                5253.000000   
mean       25.887181                  200.754425                   5.327846   
std      1333.402341                13801.017341                 282.910680   
min     -3206.130000                   -1.000000                  -1.000000   
25%        -0.595000                    0.000000                   0.000000   
50%         0.000000                    0.000000                   0.155000   
75%         1.739000                    0.397000                   0.485000   
max     68287.150000              1000000.000000               20469.480000   

       AnalystTargetPrice    TrailingPE     ForwardPE  PriceToSalesRatioTTM  \
count         3917.000000   5253.000000   5253.000000           5253.000000   
mean           195.368609     27.846999     28.157959            800.345937   
std           7894.265600    252.003342    234.959187          24371.170263   
min              0.000000      0.000000      0.000000              0.000000   
25%             13.670000      0.000000      0.000000              0.181000   
50%             29.670000      0.000000      0.000000              1.874000   
75%             67.170000     17.030000     14.140000              5.800000   
max         494000.000000  10130.000000  10000.000000        1000000.000000   

       PriceToBookRatio     EVToRevenue    EVToEBITDA         Beta  \
count   

In [51]:
clean_df['Industry'].value_counts()[:50].sum()

4408

# EPS Work 

Current formula messes up dates in the index. I tried to use [26:] as a dirty fix but it ended up leaving duplicates for more recent years

For the time being, focus on setting up a clustering model before pulling additional data

In [ ]:
# Set empty dataframe and list
primary = pd.DataFrame([pd.to_datetime(-i, format='%Y').year for i in range(-2021,-1995)], columns=['Year'])
primary.set_index('Year', inplace=True)

no_EPS_data = []

In [ ]:
## Download Earnings Per Share Data for every stock
%%time
counter = 0
for i in clean_df.Symbol:
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={i}&apikey={key_alpha}'
    r = requests.get(url)
    data = r.json()
    
    counter += 1
    if counter%250 == 0:
        print(f"{counter} completed")

    if data['annualEarnings'] == []:
        no_EPS_data.append(i)

    else:
        test = pd.DataFrame(data['annualEarnings'])

        test.columns = ['fiscalDateEnding', i]

        test['Year'] = pd.to_datetime(test['fiscalDateEnding'], format= '%Y-%m-%d').dt.year

        test.drop(['fiscalDateEnding'], axis=1, inplace=True)

        test.set_index('Year', inplace=True)

        primary = primary.merge(test, how='left', left_on='Year', right_on='Year')[-26:]

In [ ]:
# primary.to_csv('data/EPS_data.csv')

# Cash Flow Data

In [53]:
url = 'https://www.alphavantage.co/query?function=CASH_FLOW&symbol=IBM&apikey='+ key_alpha
r = requests.get(url)
cash_flow_IBM = r.json()


print(cash_flow_IBM)

{'symbol': 'IBM', 'annualReports': [{'fiscalDateEnding': '2020-12-31', 'reportedCurrency': 'USD', 'operatingCashflow': '18197000000', 'paymentsForOperatingActivities': '3406000000', 'proceedsFromOperatingActivities': 'None', 'changeInOperatingLiabilities': '138000000', 'changeInOperatingAssets': '-5088000000', 'depreciationDepletionAndAmortization': '6695000000', 'capitalExpenditures': '2618000000', 'changeInReceivables': '-5297000000', 'changeInInventory': '209000000', 'profitLoss': '5590000000', 'cashflowFromInvestment': '-3028000000', 'cashflowFromFinancing': '-9721000000', 'proceedsFromRepaymentsOfShortTermDebt': '-853000000', 'paymentsForRepurchaseOfCommonStock': 'None', 'paymentsForRepurchaseOfEquity': 'None', 'paymentsForRepurchaseOfPreferredStock': 'None', 'dividendPayout': '5797000000', 'dividendPayoutCommonStock': '5797000000', 'dividendPayoutPreferredStock': 'None', 'proceedsFromIssuanceOfCommonStock': 'None', 'proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet': '105

## Archive

In [ ]:
# if stock split is greater than a certain ratio.
# Dropped because I don't see it being important
factor = []
for i in base_df['LastSplitFactor']:
    if i == 'None':
        factor.append(0)
    else:
        temp = i.split(':')
        if int(temp[1]) == 0:
            factor.append(0)
        else:
            if int(temp[0])/int(temp[1]) > 4:
                factor.append(1)